In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import pickle
import os

In [3]:
config_path = "./config.yml"
with open(config_path, "r") as handle:
    config = yaml.load(handle, Loader=yaml.FullLoader)
resultsdir = config["resultsdir"]
precision_scores_path = os.path.join(resultsdir, "precision_scores.pkl")
recall_scores_path = os.path.join(resultsdir, "recall_scores.pkl")

with open(precision_scores_path, "rb") as handle:
    prec_data = pickle.load(handle)

with open(recall_scores_path, "rb") as handle:
    rec_data = pickle.load(handle)

colors = config["colors"]
methods = config["methods"]
datpath = config["datapath"]

sample_ids = os.listdir(os.path.join(datpath, methods[0]))
sample_ids = [s.split(".")[0] for s in sample_ids]
radii = config["radii"]


In [4]:
sns.set(
    font='Franklin Gothic Book',
    rc={'axes.axisbelow': False,
        'axes.edgecolor': 'grey',
        'axes.facecolor': 'None',
        'axes.grid': False,
        'axes.labelcolor': 'grey',
        'axes.spines.right': True,
        'axes.spines.top': True,
        'figure.facecolor': 'white',
        'lines.solid_capstyle': 'round',
        'patch.edgecolor': 'w',
        'patch.force_edgecolor': True,
        'text.color': 'grey',
        'xtick.bottom': False,
        'xtick.color': 'grey',
        'xtick.direction': 'out',
        'xtick.top': False,
        'ytick.color': 'grey',
        'ytick.direction': 'out',
        'ytick.left': False,
        'ytick.right': False}
)
sns.set_context("notebook", rc={"font.size":18,
                                "axes.titlesize":18,
                                "axes.labelsize":18})

names = {
    "mesmer": "Mesmer",
    "stardist": "StarDist",
    "unet": "U-Net",
    "cellpose": "CellPose",
    "maskrcnn": "Mask-RCNN"
}

In [5]:
points = dict((m, []) for m in methods)

for sid in sample_ids:
    for i, rad in enumerate(radii):
        for m in methods:
            p = prec_data[sid][rad][m]
            r = rec_data[sid][rad][m]
            points[m].append((r, p))

KeyError: 'mesmer'

In [7]:
prec_data

{'OHSU_TMA1_004-A1': {2: {},
  4: {},
  6: {},
  8: {},
  10: {},
  12: {},
  14: {},
  16: {'mesmer': 0.9148936170212766,
   'stardist': 0.9935897435897436,
   'maskrcnn': 1.0,
   'cellpose': 0.9705882352941176,
   'unet': 0.926829268292683}},
 'OHSU_TMA1_004-B9': {2: {},
  4: {},
  6: {},
  8: {},
  10: {},
  12: {},
  14: {},
  16: {'mesmer': 0.9661016949152542,
   'stardist': 0.9705882352941176,
   'maskrcnn': 0.979381443298969,
   'cellpose': 0.9519230769230769,
   'unet': 0.9482758620689655}},
 'OHSU_TMA1_004-E4': {2: {},
  4: {},
  6: {},
  8: {},
  10: {},
  12: {},
  14: {},
  16: {'mesmer': 0.9285714285714286,
   'stardist': 0.9583333333333334,
   'maskrcnn': 0.9629629629629629,
   'cellpose': 0.9285714285714286,
   'unet': 0.9696969696969697}},
 'OHSU_TMA1_004-D1': {2: {},
  4: {},
  6: {},
  8: {},
  10: {},
  12: {},
  14: {},
  16: {'mesmer': 0.9312320916905444,
   'stardist': 0.9037900874635568,
   'maskrcnn': 0.8023952095808383,
   'cellpose': 0.9144542772861357,
   'un

In [ ]:
fig, ax = plt.subplots(2, 3, sharex=True, sharey=True)
fig.set_figheight(6.5)
fig.set_figwidth(14.5)

fig.text(0.5, 0.03, 'Recall', ha='center')
fig.text(0.07, 0.5, 'Precision', va='center', rotation='vertical')

for i, m in enumerate(methods):
    row = 0 if i < 3 else 1
    c = colors[i]
    if i >= 3:
        i -= 3
    ax[row][i].set_xlim(0.3, 1)
    ax[row][i].set_ylim(0.5, 1)
    ax[row][i].set_title(names[m])
    xs = list(map(lambda x : x[0], points[m]))
    ys = list(map(lambda x : x[1], points[m]))
    sns.kdeplot(x=xs, y=ys, colors=c, clip=(0,1), ax=ax[row][i], **{"linewidths":0.9})
    for x, y in list(zip(xs, ys)):
        ax[row][i].scatter(x, y, color=c, alpha=[0.15])
        ax[1][2].scatter(x, y, color=colors[i], alpha=[0.3])

In [ ]:
offsets = [-0.5, -0.25, 0, 0.25, 0.5]

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

all_precs = dict((rad, []) for rad in radii)
all_recs = dict((rad, []) for rad in radii)

for name in methods:
    
    precisions = dict((rad, []) for rad in radii)
    recalls = dict((rad, []) for rad in radii)
    
    for sid in sample_ids:
        
        for i, rad in enumerate(radii):

            p = prec_data[sid][rad][name]
            r = rec_data[sid][rad][name]
            precisions[rad].append(p)
            recalls[rad].append(r)
    
    for i, rad in enumerate(radii):
        all_precs[rad].append(precisions[rad])
        all_recs[rad].append(recalls[rad])

In [ ]:
fig = plt.figure(figsize=(24, 8))

for rad, data in all_precs.items():
    
    for i, d in enumerate(data):

        bpl = plt.boxplot(d, positions=[offsets[i] + rad], patch_artist=True, sym='', widths=0.1)
        set_box_color(bpl, colors[i])
        plt.ylabel("Precision")
        plt.xlabel("Dilation Radius")
        plt.xticks(radii, list(radii))
        
plt.plot([], c=colors[0], label=names[methods[0]])
plt.plot([], c=colors[1], label=names[methods[1]])
plt.plot([], c=colors[2], label=names[methods[2]])
plt.plot([], c=colors[3], label=names[methods[3]])
plt.plot([], c=colors[4], label=names[methods[4]])

In [ ]:
fig = plt.figure(figsize=(24, 8))

for rad, data in all_recs.items():
    
    for i, d in enumerate(data):

        bpl = plt.boxplot(d, positions=[offsets[i] + rad], patch_artist=True, sym='', widths=0.1)
        set_box_color(bpl, colors[i])
        plt.ylabel("Recall")
        plt.xlabel("Dilation Radius")
        plt.xticks(radii, list(radii))
        
plt.plot([], c=colors[0], label=names[methods[0]])
plt.plot([], c=colors[1], label=names[methods[1]])
plt.plot([], c=colors[2], label=names[methods[2]])
plt.plot([], c=colors[3], label=names[methods[3]])
plt.plot([], c=colors[4], label=names[methods[4]])

plt.legend(loc="lower right")